In [60]:
import pandas as pd

from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.chart import BarChart, Reference

#reading the excel file
df = pd.read_excel('sample_pivot.xlsx', parse_dates=['Date'])

#print first 5 rows
print(df.head())

        Date Region                 Type  Units  Sales
0 2020-07-11   East  Children's Clothing   18.0    306
1 2020-09-23  North  Children's Clothing   14.0    448
2 2020-04-02  South     Women's Clothing   17.0    425
3 2020-02-28   East  Children's Clothing   26.0    832
4 2020-03-19   West     Women's Clothing    3.0     33


In [61]:
filtered = df[df['Region'] == 'West']
quarterly_sales = pd.pivot_table(filtered, index = filtered['Date'].dt.quarter, columns = 'Type', values = 'Sales', aggfunc='sum')

#create pivot table to view region west
print("Quarterly Sales Pivot Table:")
print(quarterly_sales.head())

Quarterly Sales Pivot Table:
Type  Children's Clothing  Men's Clothing  Women's Clothing
Date                                                       
1                    7264            6920              4816
2                    4711            2762              3389
3                    2218            2844              6505
4                    5989            6551              7507


In [62]:
f = pd.ExcelWriter('quarterly_sales.xlsx', engine='xlsxwriter')
quarterly_sales.to_excel(f, sheet_name = 'Quarterly Sales', startrow = 3)
f.save()

In [63]:
# Load the Workbook
wb = load_workbook('quarterly_sales.xlsx')
sheet1 = wb['Quarterly Sales']

# Format the First Sheet
sheet1['A1'] = 'Quarterly Sales'
sheet1['A2'] = 'Pa Vang'
sheet1['A4'] = 'Quarter'

sheet1['A1'].style = 'Title'
sheet1['A2'].style = 'Headline 2'

for i in range(5, 9):
    sheet1[f'B{i}'].style='Currency'
    sheet1[f'C{i}'].style='Currency'
    sheet1[f'D{i}'].style='Currency'

# Add a Bar Chart
bar_chart = BarChart()
data = Reference(sheet1, min_col=2, max_col=4, min_row=4, max_row=8)
categories = Reference(sheet1, min_col=1, max_col=1, min_row=5, max_row=8)
bar_chart.add_data(data, titles_from_data=True)
bar_chart.set_categories(categories)
sheet1.add_chart(bar_chart, "F4")

bar_chart.title = 'Sales by Type'
bar_chart.style = 3
wb.save('quarterly_sales.xlsx')

In [64]:
# Get Region Names
regions = list(df['Region'].unique())

# Loop Over All Regions
for region in regions:
    filtered = df[df['Region'] == f'{region}']
    quarterly_sales = pd.pivot_table(filtered, index = filtered['Date'].dt.quarter, columns = 'Type', values = 'Sales', aggfunc='sum')
    file_path = f"{region}.xlsx"
    quarterly_sales.to_excel(file_path, sheet_name = 'Quarterly Sales', startrow = 3)
    
    wb = load_workbook(file_path)
    sheet1 = wb['Quarterly Sales']
    
    sheet1['A1'] = 'Quarterly Sales'
    sheet1['A2'] = 'Pa Vang'
    sheet1['A4'] = 'Quarter'

    sheet1['A1'].style = 'Title'
    sheet1['A2'].style = 'Headline 2'

    for i in range(5, 10):
        sheet1[f'B{i}'].style='Currency'
        sheet1[f'C{i}'].style='Currency'
        sheet1[f'D{i}'].style='Currency'

    bar_chart = BarChart()
    data = Reference(sheet1, min_col=2, max_col=4, min_row=4, max_row=8)
    categories = Reference(sheet1, min_col=1, max_col=1, min_row=5, max_row=8)
    bar_chart.add_data(data, titles_from_data=True)
    bar_chart.set_categories(categories)
    sheet1.add_chart(bar_chart, "F4")

    bar_chart.title = 'Sales by Type'
    bar_chart.style = 3
    wb.save(file_path)